2- read dataset

In [1]:
import json
from itertools import chain

# Load the dataset from a JSON file
with open("train_data.json", "r") as file:
    data = json.load(file)  

# 1. Number of users
num_users = len(data)

# 2. Number of unique movies
all_movies = list(chain(*data))  
unique_movies = set(all_movies)  # Get unique movie IDs
num_movies = len(unique_movies)

# Display results
print(f"Train Dataset")
print(f"Number of users: {num_users}")
print(f"Number of unique movies: {num_movies}")


Train Dataset
Number of users: 6022
Number of unique movies: 3043


3) Implement model

In this subsection, you must implement the NGCF1 model, evaluate your models with
the MovieLens-1M dataset, report the Recall and NDCG2 metric, and explain your
loss-epoch plots.

In [3]:
import json
import numpy as np

# Load datasets from JSON files
with open('train_data.json', 'r') as file:
    train_data = json.load(file)

with open('test_data.json', 'r') as file:
    test_data = json.load(file)

with open('validation_data.json', 'r') as file:
    valid_data = json.load(file)

# Prepare the user-item interaction data for each dataset
def prepare_interaction_matrix(data):
    num_users = len(data)
    num_movies = max(np.max(user_items) for user_items in data) + 1  # Assuming movie IDs are 0-indexed
    # all_movies = list(chain(*data))  # Flatten the lists
    # unique_movies = set(all_movies)  # Get unique movie IDs
    # num_movies = len(unique_movies)
    
    interaction_matrix = np.zeros((num_users, num_movies), dtype=np.float32)
    for user_id, items in enumerate(data):
        interaction_matrix[user_id, items] = 1  # Indicating interaction with the movie
    return interaction_matrix

# Prepare interaction matrices
train_matrix = prepare_interaction_matrix(train_data)
test_matrix = prepare_interaction_matrix(test_data)
valid_matrix = prepare_interaction_matrix(valid_data)




print(f"Train data shape: {train_matrix.shape}")
print(f"Test data shape: {test_matrix.shape}")
print(f"Validation data shape: {valid_matrix.shape}")


Train data shape: (6022, 3043)
Test data shape: (6022, 3043)
Validation data shape: (6022, 3043)


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F

class NGCF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=64):
        super(NGCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        
        # Additional layers for the Graph Neural Network (GNN) approach (optional)
        self.gcn_layer = nn.Linear(embedding_dim * 2, embedding_dim)
        
    def forward(self, users, items):
        user_emb = self.user_embedding(users)
        item_emb = self.item_embedding(items)
        
        # Concatenate user and item embeddings for interaction
        combined_emb = torch.cat([user_emb, item_emb], dim=-1)
        
        # Apply the GCN layer followed by a ReLU activation function to produce the interaction embeddings.(here a simple fully connected layer)
        interaction_emb = F.relu(self.gcn_layer(combined_emb))
        
        return interaction_emb
    
    def predict(self, users, items):
        # Compute the predicted scores for a user-item pair
        user_emb = self.user_embedding(users)
        item_emb = self.item_embedding(items)
        
        # Dot product between user and item embeddings to get a scalar prediction score
        scores = torch.sum(user_emb * item_emb, dim=-1)  # Scalar score for each user-item pair
        return scores

# Initialize the model, loss function, and optimizer
model = NGCF(num_users, num_movies)
criterion = nn.BCEWithLogitsLoss() # is used for binary classification tasks (a user has interacted with an item or not.)
optimizer = optim.Adam(model.parameters(), lr=0.001) #uses the Adam algorithm to update the model's parameters


In [ ]:
import torch
import numpy as np
import torch.optim as optim

train_tensor = torch.tensor(train_matrix, dtype=torch.float32)
test_tensor = torch.tensor(test_matrix, dtype=torch.float32)
valid_tensor = torch.tensor(valid_matrix, dtype=torch.float32)



# Helper function for training
def train_model(model, train_data, epochs=20):
    train_loss = []
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        # Create mini-batches of user-item pairs
        for user_id in range(train_data.shape[0]):
            user_interactions = np.nonzero(train_data[user_id])[0]
            
            if len(user_interactions) == 0:
                continue
            
            user_tensor = torch.tensor([user_id] * len(user_interactions), dtype=torch.long)
            item_tensor = torch.tensor(user_interactions, dtype=torch.long)
            labels = torch.tensor(np.ones(len(user_interactions)), dtype=torch.float32)  # Positive interactions
            
            optimizer.zero_grad()
            
            # Forward pass: Get model predictions
            preds = model.predict(user_tensor, item_tensor)  # Predict the scores
            
            # Calculate loss
            loss = criterion(preds, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / train_data.shape[0]
        train_loss.append(avg_loss)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
    
    return train_loss

# Train the model
train_loss = train_model(model, train_tensor)


C:\Users\Daniyal\AppData\Local\Temp\ipykernel_15272\2404724550.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item_tensor = torch.tensor(user_interactions, dtype=torch.long)


Epoch 1/40, Loss: 3.1201
Epoch 2/40, Loss: 2.0306
Epoch 3/40, Loss: 1.2204
Epoch 4/40, Loss: 0.6962
Epoch 5/40, Loss: 0.3771
Epoch 6/40, Loss: 0.1954
Epoch 7/40, Loss: 0.0980
Epoch 8/40, Loss: 0.0476
Epoch 9/40, Loss: 0.0223
Epoch 10/40, Loss: 0.0099
Epoch 11/40, Loss: 0.0040
Epoch 12/40, Loss: 0.0017
Epoch 13/40, Loss: 0.0007
Epoch 14/40, Loss: 0.0002
Epoch 15/40, Loss: 0.0000
Epoch 16/40, Loss: 0.0000
Epoch 17/40, Loss: 0.0000
Epoch 18/40, Loss: 0.0000
Epoch 19/40, Loss: 0.0000
Epoch 20/40, Loss: 0.0000
Epoch 21/40, Loss: 0.0000
Epoch 22/40, Loss: 0.0000
Epoch 23/40, Loss: 0.0000
Epoch 24/40, Loss: 0.0000
Epoch 25/40, Loss: 0.0000
Epoch 26/40, Loss: 0.0000
Epoch 27/40, Loss: 0.0000
Epoch 28/40, Loss: 0.0000
Epoch 29/40, Loss: 0.0000
Epoch 30/40, Loss: 0.0000
Epoch 31/40, Loss: 0.0000
Epoch 32/40, Loss: 0.0000
Epoch 33/40, Loss: 0.0000
Epoch 34/40, Loss: 0.0000
Epoch 35/40, Loss: 0.0000
Epoch 36/40, Loss: 0.0000
Epoch 37/40, Loss: 0.0000
Epoch 38/40, Loss: 0.0000
Epoch 39/40, Loss: 0.

In [5]:
def recall_at_k(predictions, ground_truth, k=10):
    # Ensure we only get the top k items, handle case where there are fewer than k items
    top_k_items = predictions.topk(min(k, predictions.size(0))).indices
    recall = sum([1 if item in top_k_items[i].cpu().numpy() else 0 for i, item in enumerate(ground_truth)])
    return recall / len(ground_truth)

def ndcg_at_k(predictions, ground_truth, k=10):
    # Ensure we only get the top k items, handle case where there are fewer than k items
    top_k_items = predictions.topk(min(k, predictions.size(0))).indices
    dcg = sum([1 / np.log2(i + 2) if item in top_k_items[i].cpu().numpy() else 0 for i, item in enumerate(ground_truth)])
    idcg = sum([1 / np.log2(i + 2) for i in range(min(k, len(ground_truth)))])  # Ideal DCG
    return dcg / idcg if idcg > 0 else 0

def evaluate_model(model, test_data, k=10):
    model.eval()
    
    recall_list = []
    ndcg_list = []
    
    for user_id in range(test_data.shape[0]):  # Use .shape on NumPy array or PyTorch tensor
        user_interactions = np.nonzero(test_data[user_id])[0]
        
        if len(user_interactions) == 0:
            continue
        
        user_tensor = torch.tensor([user_id] * len(user_interactions), dtype=torch.long)
        item_tensor = torch.tensor(user_interactions, dtype=torch.long)
        
        with torch.no_grad():
            predictions = model.predict(user_tensor, item_tensor)
        
        recall = recall_at_k(predictions, user_interactions, k)
        ndcg = ndcg_at_k(predictions, user_interactions, k)
        
        recall_list.append(recall)
        ndcg_list.append(ndcg)
    
    avg_recall = np.mean(recall_list)
    avg_ndcg = np.mean(ndcg_list)
    
    return avg_recall, avg_ndcg

# Assuming test_tensor is the correct format
recall, ndcg = evaluate_model(model, test_tensor, k=10)  # Use the PyTorch tensor here
print(f"Recall@10: {recall:.4f}")
print(f"NDCG@10: {ndcg:.4f}")


C:\Users\Daniyal\AppData\Local\Temp\ipykernel_15272\3451842833.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item_tensor = torch.tensor(user_interactions, dtype=torch.long)


Recall@10: 0.0387
NDCG@10: 0.0387


In [6]:
import plotly.graph_objs as go
import plotly.io as pio

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(len(train_loss))),
    y=train_loss,
    mode='lines+markers',
    name='Train Loss'
))

# Update layout
fig.update_layout(
    title='Training Loss Over Epochs',
    xaxis_title='Epoch',
    yaxis_title='Loss'
)

# Show the plot
pio.show(fig)

8

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class NGCF_LSTM(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=64, lstm_hidden_dim=128, num_layers=3):
        super(NGCF_LSTM, self).__init__()
        # User and item embeddings
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        
        # LSTM for combining embeddings from different layers
        self.lstm = nn.LSTM(input_size=embedding_dim * 2, hidden_size=lstm_hidden_dim, num_layers=num_layers, batch_first=True)
        
        # Final projection layer to reduce to embedding_dim
        self.fc = nn.Linear(lstm_hidden_dim, embedding_dim)
        
    def forward(self, users, items, neighbors):
        # Initialize embeddings
        user_emb = self.user_embedding(users)
        item_emb = self.item_embedding(items)
        
        # Combine embeddings of users and items for interaction
        combined_emb = torch.cat([user_emb, item_emb], dim=-1)
        
        # Propagate embeddings through different layers
        all_embeddings = [combined_emb]  # Start with the initial embeddings
        
        # Iterate over multiple layers to generate higher-order embeddings
        for _ in range(3):  # Assume 3 layers for illustration
            new_emb = self.propagate_layer(all_embeddings[-1], neighbors)
            all_embeddings.append(new_emb)
        
        # Stack embeddings from all layers (first-order, second-order, etc.)
        layer_embeddings = torch.stack(all_embeddings, dim=1)
        
        # LSTM to aggregate embeddings across layers
        lstm_out, _ = self.lstm(layer_embeddings)
        
        # Get the last LSTM output for each user-item pair
        lstm_out = lstm_out[:, -1, :]
        
        # Final projection to reduce dimensions
        output = self.fc(lstm_out)
        
        return output
    
    def propagate_layer(self, embeddings, neighbors):
        # Simple propagation function to update embeddings (similar to NGCF's propagation)
        neighbor_embs = embeddings[neighbors]  # Gather embeddings of neighbors
        updated_emb = torch.mean(neighbor_embs, dim=1)  # Simple average aggregation
        return updated_emb

    def predict(self, users, items):
        # Compute the predicted scores for a user-item pair
        user_emb = self.user_embedding(users)
        item_emb = self.item_embedding(items)
        
        # Dot product between user and item embeddings to get a scalar prediction score
        scores = torch.sum(user_emb * item_emb, dim=-1)  # Scalar score for each user-item pair
        return scores



In [14]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

################################################################################
# 1. Define the NGCF_LSTM Model
################################################################################

class NGCF_LSTM(nn.Module):
    """
    An example NGCF model that replaces the typical layer aggregation
    with an LSTM-based approach. This is a simplified illustration:
      - user_embedding / item_embedding: standard embeddings
      - LSTM: aggregates embeddings across 'layers'
      - fc: final projection

    Args:
        num_users (int):    Number of users
        num_items (int):    Number of items
        embedding_dim (int):Dimension of user/item embeddings
        lstm_hidden_dim(int): Hidden dimension for LSTM
        num_layers (int):   Number of LSTM layers
    """
    def __init__(self, num_users, num_items, embedding_dim=64,
                 lstm_hidden_dim=128, num_layers=3):
        super(NGCF_LSTM, self).__init__()
        
        # User and item embeddings
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        
        # LSTM to combine 'layer embeddings'
        # We'll assume input_size = embedding_dim*2 (user_emb + item_emb),
        # but we do a short multi-layer approach for demonstration.
        self.lstm = nn.LSTM(
            input_size = embedding_dim * 2,
            hidden_size = lstm_hidden_dim,
            num_layers = num_layers,
            batch_first = True
        )
        
        # Final projection layer (maps from lstm_hidden_dim -> embedding_dim)
        self.fc = nn.Linear(lstm_hidden_dim, embedding_dim)
        
    def forward(self, users, items, neighbors):
        """
        Forward pass. The 'neighbors' tensor is not really used here
        except to fit the signature. In a real NGCF you might gather
        neighbor embeddings based on adjacency, etc.
        
        Args:
            users (Tensor): user IDs shape [batch_size]
            items (Tensor): item IDs shape [batch_size]
            neighbors (Tensor): dummy or real neighbor IDs shape [batch_size, ...] (depends on your design)
        Returns:
            Tensor of shape [batch_size, embedding_dim], the final representation
        """
        # 1) Basic user/item embeddings
        user_emb = self.user_embedding(users)   # [batch_size, embedding_dim]
        item_emb = self.item_embedding(items)   # [batch_size, embedding_dim]
        
        # 2) Combine them in a naive way: just cat user_emb and item_emb
        #    so shape becomes [batch_size, embedding_dim * 2]
        combined_emb = torch.cat([user_emb, item_emb], dim=-1)
        
        # For demonstration, we pretend we have multiple "layers" of embeddings
        # to feed the LSTM. We'll create a small sequence by replicating the
        # combined_emb some number of times. In practice, you'd gather
        # embeddings from multiple GCN propagation layers, etc.
        # shape for LSTM: [batch_size, seq_len, input_size]
        # We'll do a contrived "seq_len=3" to emulate "3 layers"
        repeated_seq = torch.stack([combined_emb, combined_emb, combined_emb], dim=1)
        # shape = [batch_size, 3, embedding_dim*2]
        
        # 3) Pass that sequence through the LSTM
        lstm_out, _ = self.lstm(repeated_seq)   # [batch_size, seq_len, lstm_hidden_dim]
        
        # We'll take the final step (seq_len-1) hidden states for each sample
        # shape becomes [batch_size, lstm_hidden_dim]
        final_state = lstm_out[:, -1, :]
        
        # 4) Project down to embedding_dim
        output = self.fc(final_state)  # [batch_size, embedding_dim]
        return output
    
    def predict(self, users, items):
        """
        A typical "predict" method that returns a scalar score for each pair
        by dot-product of user and item embeddings. This is used in standard CF.
        """
        user_emb = self.user_embedding(users)
        item_emb = self.item_embedding(items)
        scores = torch.sum(user_emb * item_emb, dim=-1)  # scalar per pair
        return scores



################################################################################
# 2. Example: BCEWithLogitsLoss + Adam
################################################################################


model_lstm = NGCF_LSTM(num_users, num_movies, embedding_dim=64)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model_lstm.parameters(), lr=0.001)


################################################################################
# 3. Data Preparation
################################################################################
# We'll assume 'train_data' is a NumPy 2D array of shape [num_users, num_items],
# with 1's for interactions and 0's for no interaction.

# For example, let's create a synthetic data matrix (very sparse):
# train_data = np.zeros((num_users, num_movies), dtype=np.float32)
# # Put some random interactions:
# for u in range(num_users):
#     # randomly pick 2 items to "interact"
#     items_chosen = np.random.choice(num_movies, 2, replace=False)
#     train_data[u, items_chosen] = 1.


################################################################################
# 4. The Revised Training Function
################################################################################

def train_lstm_model(model, train_data, epochs=20):
    """
    Matches the structure of your old train_model function:
    Iterates through each user -> finds items -> passes batch to the model.
    """
    train_loss_values = []
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        
        # Loop over each user in the train_data
        for user_id in range(train_data.shape[0]):
            # Find items the user has interacted with
            user_interactions = np.nonzero(train_data[user_id])[0]  # array of item indices
            
            if len(user_interactions) == 0:
                continue
            
            # Tensors
            user_tensor = torch.tensor([user_id]*len(user_interactions), dtype=torch.long)
            item_tensor = torch.tensor(user_interactions, dtype=torch.long)
            # Dummy neighbors (since the model signature expects it)
            neighbors_tensor = torch.zeros(len(user_interactions), dtype=torch.long)
            
            # Labels: assume all are positive = 1
            labels = torch.ones(len(user_interactions), dtype=torch.float32)
            
            # Forward pass
            optimizer.zero_grad()
            preds = model(user_tensor, item_tensor, neighbors_tensor)  # shape [batch_size, embedding_dim]
            
            # But BCEWithLogitsLoss expects shape [batch_size] (raw scores)
            # We must reduce 'preds' to a single scalar per user-item pair
            # Let's do a quick fix: We can do a dot-product approach
            # or simply pick one dimension. For demonstration, let's just do:
            preds_scalar = preds.mean(dim=1)  # shape [batch_size]
            
            loss = criterion(preds_scalar, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / train_data.shape[0]
        train_loss_values.append(avg_loss)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
    
    return train_loss_values

################################################################################
# 5. Training & Testing
################################################################################


train_tensor = torch.tensor(train_matrix, dtype=torch.float32)
test_tensor = torch.tensor(test_matrix, dtype=torch.float32)
valid_tensor = torch.tensor(valid_matrix, dtype=torch.float32)

# Train the model
train_losses = train_lstm_model(model_lstm, train_tensor, epochs=20)

# Suppose you want to do a "predict" for user=0, item=10:
with torch.no_grad():
    test_score = model_lstm.predict(
        torch.tensor([0], dtype=torch.long),
        torch.tensor([10], dtype=torch.long)
    )
    print("Test Score (user=0, item=10):", test_score.item())



C:\Users\Daniyal\AppData\Local\Temp\ipykernel_15272\2838234758.py:150: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 1/20, Loss: 0.0027
Epoch 2/20, Loss: 0.0000
Epoch 3/20, Loss: 0.0000
Epoch 4/20, Loss: 0.0000
Epoch 5/20, Loss: 0.0000
Epoch 6/20, Loss: 0.0000
Epoch 7/20, Loss: 0.0000
Epoch 8/20, Loss: 0.0000
Epoch 9/20, Loss: 0.0000
Epoch 10/20, Loss: 0.0000
Epoch 11/20, Loss: 0.0000
Epoch 12/20, Loss: 0.0000
Epoch 13/20, Loss: 0.0000
Epoch 14/20, Loss: 0.0000
Epoch 15/20, Loss: 0.0000
Epoch 16/20, Loss: 0.0000
Epoch 17/20, Loss: 0.0000
Epoch 18/20, Loss: 0.0000
Epoch 19/20, Loss: 0.0000
Epoch 20/20, Loss: 0.0000
Test Score (user=0, item=10): -2.552618980407715


In [15]:
recall, ndcg, f1 = evaluate_model(model_lstm, test_tensor, k=10)  # Use the PyTorch tensor here
print(f"Recall@10: {recall:.4f}")
print(f"NDCG@10: {ndcg:.4f}")
print(f"F1@10: {f1:.4f}")


Recall@10: 0.0027
NDCG@10: 0.0012
F1@10: 0.0005


Section3

8

In [16]:
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

###############################################################################
# 1. UltraGCN Model Definition
###############################################################################
class UltraGCN(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=64):
        """
        A simplified UltraGCN-like model:
          - User and item embeddings
          - No multi-hop GCN layers (as in LightGCN/NGCF)
          - We rely on direct user–item representation learning.
        """
        super(UltraGCN, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        
        # Initialization
        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.item_embedding.weight)
    
    def forward(self, users, items):
        """
        Forward pass to produce an interaction embedding (if desired).
        In a simplified UltraGCN, you might not even need an interaction layer.
        Here, we just combine them as a demonstration.
        """
        user_emb = self.user_embedding(users)   # (batch_size, embedding_dim)
        item_emb = self.item_embedding(items)   # (batch_size, embedding_dim)
        
        # Optionally, combine user_emb and item_emb in some way
        # For example, just concatenate and apply a small transformation
        # or skip any transformation to keep it ultra-simple.
        combined_emb = torch.cat([user_emb, item_emb], dim=-1)
        return combined_emb
    
    def predict(self, users, items):
        """
        Compute predicted scores for a user–item pair.
        Typically, UltraGCN uses pairwise constraints or BPR-based losses.
        Here, we return the dot product for a ranking or BCE objective.
        """
        user_emb = self.user_embedding(users)
        item_emb = self.item_embedding(items)
        
        # Dot product = relevance score
        scores = torch.sum(user_emb * item_emb, dim=-1)  # shape: (batch_size,)
        return scores


###############################################################################
# 2. Loading and Preparing Data from JSON
###############################################################################
def load_data_from_json(json_path):
    """
    Suppose our JSON file is a list of lists:
        [
          [item_id_1, item_id_2, ...],  # user 0
          [item_id_1, item_id_2, ...],  # user 1
          ...
        ]
    Return a NumPy array (train_matrix) of shape (num_users, num_items).
    For a real-world scenario, you'd also create separate train/valid/test sets.
    """
    with open(json_path, 'r') as f:
        data = json.load(f)

    num_users = len(data)
    # Collect all item IDs to determine number of items
    all_items = set()
    for items in data:
        all_items.update(items)
    num_items = max(all_items) + 1

    # Build a simple binary interaction matrix
    # (user, item) = 1 if user interacted with item, else 0
    interaction_matrix = np.zeros((num_users, num_items), dtype=np.float32)
    for user_id, items in enumerate(data):
        for it in items:
            interaction_matrix[user_id, it] = 1.0

    return interaction_matrix


###############################################################################
# 3. Training Setup (Loss and Helper Functions)
###############################################################################
criterion = nn.BCEWithLogitsLoss()

def train_model(model, train_data, optimizer, epochs=20, device='cpu'):
    """
    A simple training loop that treats each user–item interaction as a positive instance
    and does not explicitly sample negatives. For a more robust approach,
    you would sample negative items as well (e.g., BPR or advanced sampling).
    """
    model.to(device)
    train_loss_history = []
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        
        # We'll iterate user by user, as in your example
        num_users = train_data.shape[0]
        for user_id in range(num_users):
            # Positive interactions = items with value 1
            user_interactions = np.nonzero(train_data[user_id])[0]  # list of item indices
            
            # If a user has no interactions, skip
            if len(user_interactions) == 0:
                continue
            
            # Build the input tensors
            user_tensor = torch.tensor([user_id]*len(user_interactions), dtype=torch.long, device=device)
            item_tensor = torch.tensor(user_interactions, dtype=torch.long, device=device)
            
            # Labels for positive interactions = 1
            labels = torch.ones(len(user_interactions), dtype=torch.float32, device=device)
            
            # Reset gradients
            optimizer.zero_grad()
            
            # Predict scores
            preds = model.predict(user_tensor, item_tensor)
            # Compute BCE loss
            loss = criterion(preds, labels)
            
            # Backprop
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / num_users
        train_loss_history.append(avg_loss)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
    
    return train_loss_history

def recall_at_k(predictions, ground_truth, k=20):
    """
    predictions: Tensor of shape [N], predicted relevance scores for each item.
    ground_truth: List (or array) of actual relevant item indices. 
      - In this simplified example, we might be calling it with the same items we predicted on, 
        so we must adapt the logic accordingly.
    This function may need adaptation depending on how you structure your evaluation.
    """
    _, topk_indices = torch.topk(predictions, k)  # top k predicted
    topk_indices = topk_indices.cpu().numpy()
    
    # Count how many ground_truth items appear in the top k
    hits = 0
    for item in ground_truth:
        # If item is in topk_indices
        if item in topk_indices:
            hits += 1
    return hits / len(ground_truth) if len(ground_truth) > 0 else 0.0

def ndcg_at_k(predictions, ground_truth, k=20):
    """
    Compute nDCG@k for the predicted scores vs ground truth items.
    """
    _, topk_indices = torch.topk(predictions, k)
    topk_indices = topk_indices.cpu().numpy()
    
    dcg = 0.0
    for i, idx in enumerate(topk_indices):
        if idx in ground_truth:
            # rank position is i, DCG is 1 / log2(rank+2)
            dcg += 1.0 / np.log2(i+2)
    # ideal DCG = sum of 1/log2(rank+2) for each relevant item in the top k 
    # (in the best scenario, all ground_truth appear at the top)
    # But for simplicity, we assume all ground_truth items are relevant.
    idcg = 0.0
    for i in range(min(len(ground_truth), k)):
        idcg += 1.0 / np.log2(i+2)
    
    return (dcg / idcg) if idcg > 0 else 0.0

def precision_at_k(predictions, ground_truth, k=20):
    """
    Compute precision@k for the predicted scores vs ground truth items.
    """
    _, topk_indices = torch.topk(predictions, k)
    topk_indices = topk_indices.cpu().numpy()
    
    # Count how many ground_truth items appear in the top k
    hits = 0
    for item in ground_truth:
        if item in topk_indices:
            hits += 1
    return hits / k if k > 0 else 0.0

def evaluate_model(model, test_data, k=20, device='cpu'):
    """
    Evaluate recall@k and ndcg@k for each user in test_data.
    For simplicity, we do the same user-by-user approach.
    """
    model.eval()
    model.to(device)
    
    recall_scores = []
    ndcg_scores = []
    f1_scores = []
    num_users = test_data.shape[0]
    
    with torch.no_grad():
        for user_id in range(num_users):
            # Items relevant to user in the test set
            ground_truth = np.nonzero(test_data[user_id])[0]
            if len(ground_truth) == 0:
                continue
            
            # We'll compute scores for all items for this user, 
            # then measure how well the top-k align with ground_truth.
            # This can be expensive if #items is large, but is okay for demonstration.
            all_items = np.arange(test_data.shape[1])
            
            user_tensor = torch.tensor([user_id]*len(all_items), dtype=torch.long, device=device)
            item_tensor = torch.tensor(all_items, dtype=torch.long, device=device)
            
            predictions = model.predict(user_tensor, item_tensor)  # shape: (num_items,)
            # Evaluate
            r = recall_at_k(predictions, ground_truth, k)
            n = ndcg_at_k(predictions, ground_truth, k)
            p = precision_at_k(predictions, ground_truth, k)
            f1 = (2 * p * r) / (p + r) if (p + r) > 0 else 0.0
            
            
            recall_scores.append(r)
            ndcg_scores.append(n)
            f1_scores.append(f1)
    
    avg_recall = np.mean(recall_scores) if len(recall_scores) > 0 else 0.0
    avg_ndcg = np.mean(ndcg_scores) if len(ndcg_scores) > 0 else 0.0
    avg_f1 = np.mean(f1_scores) if len(f1_scores) > 0 else 0.0
    
    return avg_recall, avg_ndcg, avg_f1


###############################################################################
# 4. Putting It All Together (Example Main)
###############################################################################
# 1) Load your JSON data
json_path = "train_data.json"  # Path to your JSON file
interaction_matrix_train = load_data_from_json(json_path)

# In a real project, you'd build separate train/test/val sets.
# For simplicity here, let's do an 80/20 split by user. 
# Or you might have a separate JSON for train, test, valid.
num_users = interaction_matrix_train.shape[0]
num_items = interaction_matrix_train.shape[1]

# Example: Train=first 80% of items, Test=last 20% of items for each user
# This is a simplistic approach; adapt as needed.
train_matrix = interaction_matrix_train


interaction_matrix_test = load_data_from_json("test_data.json")
test_matrix = interaction_matrix_test

# 2) Create the UltraGCN model
embedding_dim = 64
model = UltraGCN(num_users=num_users, num_items=num_items, embedding_dim=embedding_dim)

# 3) Set up optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4) Train the model
print("Training UltraGCN...")
train_model(model, train_matrix, optimizer, epochs=20, device='cpu')

# 5) Evaluate the model
print("Evaluating on test set...")
recall, ndcg, f1 = evaluate_model(model, test_matrix, k=20, device='cpu')
print(f"Recall@20 = {recall:.4f}, NDCG@20 = {ndcg:.4f}, f1@20 = {f1:.4f}")



Training UltraGCN...
Epoch 1/20, Loss: 0.6932
Epoch 2/20, Loss: 0.5155
Epoch 3/20, Loss: 0.1161
Epoch 4/20, Loss: 0.0113
Epoch 5/20, Loss: 0.0011
Epoch 6/20, Loss: 0.0001
Epoch 7/20, Loss: 0.0000
Epoch 8/20, Loss: 0.0000
Epoch 9/20, Loss: 0.0000
Epoch 10/20, Loss: 0.0000
Epoch 11/20, Loss: 0.0000
Epoch 12/20, Loss: 0.0000
Epoch 13/20, Loss: 0.0000
Epoch 14/20, Loss: 0.0000
Epoch 15/20, Loss: 0.0000
Epoch 16/20, Loss: 0.0000
Epoch 17/20, Loss: 0.0000
Epoch 18/20, Loss: 0.0000
Epoch 19/20, Loss: 0.0000
Epoch 20/20, Loss: 0.0000
Evaluating on test set...
Recall@20 = 0.0932, NDCG@20 = 0.0754, f1@20 = 0.0508
